In [2]:
%matplotlib inline

from mesa import Mesa
import argparse
from utils import Rater, load_dataset
from sklearn.tree import *
from copy import deepcopy
import time

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

parser = argparse.ArgumentParser(description='Mesa Arguments')
parser.add_argument('--env-name', default="MESA-SAC")

# SAC arguments
parser.add_argument('--policy', default="Gaussian",
                    help='Policy Type: Gaussian | Deterministic (default: Gaussian)')
parser.add_argument('--eval', type=bool, default=True,
                    help='Evaluates a policy every 10 episode (default: True)')
parser.add_argument('--gamma', type=float, default=0.99, metavar='G',
                    help='discount factor for reward (default: 0.99)')
parser.add_argument('--tau', type=float, default=0.01, metavar='G',
                    help='target smoothing coefficient(τ) (default: 0.01)')
parser.add_argument('--lr', type=float, default=0.001, metavar='G',
                    help='learning rate (default: 0.001)')
parser.add_argument('--lr_decay_steps', type=int, default=10, metavar='N',
                    help='step_size of StepLR learning rate decay scheduler (default: 10)')
parser.add_argument('--lr_decay_gamma', type=float, default=0.99, metavar='N',
                    help='gamma of StepLR learning rate decay scheduler (default: 0.99)')
parser.add_argument('--alpha', type=float, default=0.1, metavar='G',
                    help='Temperature parameter α determines the relative importance of the entropy\
                            term against the reward (default: 0.1)')
parser.add_argument('--automatic_entropy_tuning', type=bool, default=False, metavar='G',
                    help='Automaically adjust α (default: False)')
parser.add_argument('--seed', type=int, default=None, metavar='N',
                    help='random seed (default: None)')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--hidden_size', type=int, default=50, metavar='N',
                    help='hidden size (default: 50)')
parser.add_argument('--updates_per_step', type=int, default=1, metavar='N',
                    help='model updates per simul|ator step (default: 1)')
parser.add_argument('--update_steps', type=int, default=1000, metavar='N',
                    help='maximum number of steps (default: 1000)')
parser.add_argument('--start_steps', type=int, default=500, metavar='N',
                    help='Steps sampling random actions (default: 500)')
parser.add_argument('--target_update_interval', type=int, default=1, metavar='N',
                    help='Value target update per no. of updates per step (default: 1)')
parser.add_argument('--replay_size', type=int, default=1000, metavar='N',
                    help='size of replay buffer (default: 1000)')
parser.add_argument('--cuda', action="store_true", default=False,
                    help='run on CUDA (default: False)')

# MESA arguments
parser.add_argument('--dataset', type=str, default='Mammo', metavar='N',
                    help='the dataset used for meta-training (default: Mammo)')
parser.add_argument('--metric', type=str, default='aucprc', metavar='N',
                    help='the metric used for evaluate (default: aucprc)')
parser.add_argument('--reward_coefficient', type=float, default=100, metavar='N')
parser.add_argument('--num_bins', type=int, default=5, metavar='N', 
                    help='number of bins (default: 5). state-size = 2 * num_bins.')
parser.add_argument('--sigma', type=float, default=0.2, metavar='N', 
                    help='sigma of the Gaussian function used in meta-sampling (default: 0.2)')
parser.add_argument('--max_estimators', type=int, default=10, metavar='N',
                    help='maximum number of base estimators in each meta-training episode (default: 10)')
parser.add_argument('--meta_verbose', type=int, default=10, metavar='N',
                    help='number of episodes between verbose outputs. \
                    If \'full\' print log for each base estimator (default: 10)')
parser.add_argument('--meta_verbose_mean_episodes', type=int, default=25, metavar='N',
                    help='number of episodes used for compute latest mean score in verbose outputs.')
parser.add_argument('--verbose', type=bool, default=False, metavar='N',
                    help='enable verbose when ensemble fit (default: False)')
parser.add_argument('--random_state', type=int, default=None, metavar='N', 
                    help='random_state (default: None)')
parser.add_argument('--train_ir', type=float, default=1, metavar='N', 
                    help='imbalance ratio of the training set after meta-sampling (default: 1)')
parser.add_argument('--train_ratio', type=float, default=1, metavar='N', 
                    help='the ratio of the data used in meta-training. \
                    set train_ratio<1 to use a random subset for meta-training (default: 1)')

ModuleNotFoundError: No module named 'torch'

In [7]:
from datetime import datetime

# 获取当前时间
current_time = datetime.now()

# 输出当前时间
print("当前时间:", current_time, flush=True)

当前时间: 2024-09-14 00:25:57.556428


# Initialization and meta-training

In [5]:
''' Prepare the Environment '''

#  load dataset
dataset = 'Mammo'
X_train, y_train, X_valid, y_valid, X_test, y_test = load_dataset(dataset)
print("type of X_train: ", type(X_train))
print("type of y_train: ", type(y_train))
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_valid shape: ", X_valid.shape)
estimator, base_estimator = 'DT', DecisionTreeClassifier(max_depth=None)
args = parser.parse_args([])
n_estimators = args.max_estimators

# plot the class distribution
def plot_class_distribution(ax, labels, title):
    sns.countplot(data=pd.DataFrame(labels, columns=['Class']), x='Class', ax=ax)
    ax.set(title=title)
   
sns.set(style='whitegrid')
sns.set_context('talk', font_scale=1)
fig, ax = plt.subplots(figsize=(20, 6))
plot_class_distribution( 
    ax = ax, 
    labels = np.concatenate([y_train, y_valid, y_test]),
    title = f'{dataset} dataset class distribution')
plt.tight_layout(pad=1.8)
plt.show()

NameError: name 'load_dataset' is not defined

In [6]:
''' Meta-training '''

# initialize MESA
print ('Dataset: {}'.format(dataset))
mesa = Mesa(
    args=args, 
    base_estimator=base_estimator, 
    n_estimators=args.max_estimators)

# start meta-training
print ('Start meta-training of MESA ... ...')
start_time = time.clock()
mesa.meta_fit(X_train, y_train, X_valid, y_valid, X_test, y_test)
end_time = time.clock()
print ('Meta-training time: {:.3f} s'.format(end_time - start_time))

Dataset: Mammo


NameError: name 'Mesa' is not defined

# Visualize the meta-training process

In [4]:
def slide_mean(data, window_half):
    result = []
    for i in range(len(data)):
        lower_bound = max(i-window_half, 0)
        upper_bound = min(i+window_half+1, len(data)-1)
        result.append(np.mean(data[lower_bound:upper_bound]))
    return result

slide_window_half = 25
df_scores = pd.DataFrame(mesa.scores, columns=['Training', 'Validation', 'Test'])

sns.set(style='ticks')
sns.set_context('talk', font_scale=0.7)
fig = plt.figure(figsize=(12, 3.5))
for i in range(df_scores.shape[1]):
    ax = plt.subplot(1, 3, i+1)
    column = df_scores.columns[i]
    view = pd.Series(slide_mean(df_scores[column], slide_window_half))
    sns.lineplot(data=view, ax=ax)
    start_steps = args.start_steps / args.max_estimators
    ax.vlines(start_steps, view.min(), view.max(), color="orange", linestyles='dashed', linewidth=3)
    ax.text(start_steps, 
            (view.min() + view.max()) / 2, 
            'Start meta-training', 
            rotation=90, ha='center', va='center', fontsize=12)
    ax.set(title=f'{dataset} {column}', 
           xlabel='# Meta-training episodes', 
           ylabel=f'{column} AUCPRC')    
    ax.grid(axis='y')

# fig.suptitle(f'Meta-training on {dataset} dataset')
plt.tight_layout(pad=1.8)
plt.show()

# Comparison with other resampling baselines

In [6]:
import numpy as np
from collections import Counter
from sklearn.model_selection import StratifiedKFold

from baselines.canonical_resampling import Resample_classifier

def evaluate(clf, X_train, y_train, X_valid, y_valid, X_test, y_test, rater):
    score_train = rater.score(y_train, clf.predict_proba(X_train)[:,1])
    score_valid = rater.score(y_valid, clf.predict_proba(X_valid)[:,1])
    score_test = rater.score(y_test, clf.predict_proba(X_test)[:,1])
    return [score_train, score_valid, score_test]

def stratifiedKFoldTest(clf, clf_name, X, y, X_valid, y_valid, rater,
                        n_splits=4, repeat=1, random_state=None):
    scores_list, time_list = [], []
    for i_repeat in range(repeat):
        skf = StratifiedKFold(n_splits=n_splits, random_state=random_state)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            start_time = time.clock()
            if clf_name[:4] == 'MESA':
                clf.fit(X_train, y_train, X_valid, y_valid, verbose=False)
            else:
                clf.fit(X_train, y_train)
            end_time = time.clock()
            time_list.append(end_time - start_time)
            scores = evaluate(clf, X_train, y_train, X_valid, y_valid, X_test, y_test, rater)
            scores_list.append(scores)
    return scores_list, time_list

cv_params = {
    'X': np.concatenate([X_train, X_test]),
    'y': np.concatenate([y_train, y_test]),
    'X_valid': X_valid,
    'y_valid': y_valid,
    'rater': Rater('aucprc'),
    'n_splits': 4,
    'repeat': 10,
}
    
resample_names = ['ORG', 'RUS', 'NM', 'NCR', 'ENN', 
    'Tomek', 'ALLKNN', 'OSS',  'ROS', 'SMOTE', 'ADASYN', 
    'BorderSMOTE', 'SMOTEENN', 'SMOTETomek']
resample_clf_list = [Resample_classifier(resample_by=method) for method in resample_names]

def copyMesa(mesa, n_estimators=None):
    mesa_copy = deepcopy(mesa)
    if n_estimators is not None:
        mesa_copy.n_estimators = n_estimators
    return mesa_copy

from datetime import datetime
# 获取当前时间
current_time = datetime.now()
# 输出当前时间
print("当前时间:", current_time)


ensemble_size_list = [20, 10, 5]
clf_names = [f'MESA (k={size})' for size in ensemble_size_list] + resample_names
clf_list = [copyMesa(mesa, size) for size in ensemble_size_list] + resample_clf_list

df_results_list = []
for (clf, clf_name) in zip(clf_list, clf_names):
    scores_list, time_list = stratifiedKFoldTest(clf, clf_name, **cv_params)
    df_results = pd.DataFrame(scores_list, columns=['train', 'valid', 'test'])
    df_results['time'] = time_list
    df_results['method'] = clf_name
    df_results_list.append(df_results)
    info = '{:<12s} |'.format(clf_name)
    for column in ['train', 'valid', 'test']:
        info += ' {} {:.3f}-{:.3f} |'.format(
            column, df_results.mean()[column], df_results.std()[column])
    info += ' {}-fold CV ({} runs) | ave run time: {:.2f}s'.format(
        cv_params['n_splits'], cv_params['repeat'], np.mean(time_list))
    print (info)

In [7]:
# visualize the performance with error bar
df_results_all = pd.concat(df_results_list)
order_performance = df_results_all.groupby('method').mean()['test'].sort_values(ascending=False).index.tolist()
order_runtime = df_results_all.groupby('method').mean()['time'].sort_values(ascending=True).index.tolist()

order_performance = order_runtime = clf_names

# fig = plt.figure(figsize=(20, 8))
sns.set(style='whitegrid')
sns.set_context('talk', font_scale=1)
# ax = sns.barplot(x='test', y='method', data=df_results_all, ci="sd", capsize=.2, order=order)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
sns.stripplot(x='test', y='method', data=df_results_all, order=order_performance, size=4, color=".3", ax=ax1)
sns.boxplot(x='test', y='method', data=df_results_all, order=order_performance, ax=ax1)
ax1.set(xlabel='Test AUCPRC', ylabel='Method')
sns.stripplot(x='time', y='method', data=df_results_all, order=order_runtime, size=4, color=".3", ax=ax2)
sns.boxplot(x='time', y='method', data=df_results_all, order=order_runtime, ax=ax2)
ax2.set(xlabel='Run Time', ylabel='Method')

fig.suptitle('Results in {} task ({}-fold stratified CV, {} independent runs)'.format(dataset, cv_params['n_splits'], cv_params['repeat']), y=1.05)
plt.xlim(0,)
plt.tight_layout()
plt.show()